In [1]:
# NDVIを保存する
import boto3
import rasterio as rio
from rasterio.windows import Window
import matplotlib.pyplot as plt
import numpy as np
from pyproj import Transformer
import datetime
import re
import os
import common

# S3バケット名を指定
s3 = boto3.client("s3")
s3_bucket = "helios-rs-datalake"  # 自分のS3バケット名
# prefix = "Project/arkedge-sbir/Sentinel2/Paraguay-Desmochados/"  # バケット内のフォルダパス
prefix = "Project/arkedge-sbir/Sentinel2/Paraguay/"  # バケット内のフォルダパス
# prefix = "Project/arkedge-sbir/Sentinel2/Makinohara/"  # バケット内のフォルダパス
output_dir = "NDVI/"

# ファイル一覧を入手
file_list = common.get_filelist(s3, s3_bucket, prefix)

# 条件に一致するファイルを抽出
B4_files = [path for path in file_list if "B04" in path]
B8_files = [path for path in file_list if "B08" in path]

# for i in range(1): # for debug
for i in range(len(B4_files)):
    print("\r"+str(i+1) + '/' + str(len(B4_files)),end="")
    try:
        B4 = rio.open('s3://' + s3_bucket + '/' + B4_files[i])
        B8 = rio.open('s3://' + s3_bucket + '/' + B8_files[i])
        meta = B4.meta.copy()
        meta.update({
            "count": 1,  # 出力バンド数
            "dtype": "float32"
        })
        B4 = B4.read(1).astype('float')
        B8 = B8.read(1).astype('float')
        NDVI = (B8 - B4) / (B8 + B4 + 1e-6)
        outputname = os.path.basename(B4_files[i].replace('B04', 'NDVI'))
        common.s3_upload(s3, s3_bucket, prefix + output_dir + outputname, NDVI, meta)
    except Exception as e:
        print(f"Error reading files: {e}")


188/188Error reading files: list index out of range


In [ ]:
# 上記スクリプトで下記は読み込み失敗する
# Project/arkedge-sbir/Sentinel2/Paraguay-Desmochados/T21JUL_20220903T135709_B04_10m.tif
# Project/arkedge-sbir/Sentinel2/Paraguay-Desmochados/T21JUL_20221028T135711_B04_10m.tif
# Project/arkedge-sbir/Sentinel2/Paraguay-Desmochados/T21JUL_20230220T135709_B04_10m.tif
# Project/arkedge-sbir/Sentinel2/Paraguay-Desmochados/T21JUL_20230730T135709_B04_10m.tif